# Защита модели от Adversarial примеров

Работа выполнена студентом БИБ233 МИЭМ НИУ ВШЭ Коноваловым Матвеем.

Данная работа является продолжением предыдущей, в которой рассматривалось создание Adversarial примеров в задаче классификации изображений.

Атака оказалась крайне удачной, и в ходе данной работы будут рассмотрены способы это исправить.

## Основные способы защиты, рассмотренные в работе:
- Adversarial training
- нормализация входных данных
- использование небольшого dropout

Все эти изменения будут вноситься при помощи библиотеки Adversarial Robustness Toolbox.

Зададим все необходимые параметры. В MODEL_PATH укажите, куда вы положили [обученную модель из предыдущего пункта](https://disk.360.yandex.ru/d/r-5SqYBdwTfuuQ)

In [96]:
import torch
import numpy as np
import random
from pathlib import Path

IMG_SIZE = 224
MODEL_PATH = Path('models/best.pt')
DATA_DIR = Path("datasets/svhn_cls")
mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
std = np.array([0.229, 0.224, 0.225], dtype=np.float32)
LR = 1e-4
WEIGHT_DECAY = 1e-4
EPS = 8/255
EPS_STEP = 0.007
BETA = 6
MAX_PGD_ITERS = 3
SEED = 17
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

Считаем наш датасет

В качестве первого метода защиты добавим нормализацию входов в модель. Это уже должно будет снизить эффективность Adv. примеров, поскольку они существенно отличаются от нормальных примеров.

In [65]:
from art.preprocessing.standardisation_mean_std import StandardisationMeanStd

norm_proc = StandardisationMeanStd(
    mean=mean,
    std =std
)

Загрузим нашу уязвимую модель. Мы не будем обучать новую модель с нуля, вместо этого мы её дообучим

In [78]:
class YoloClsAdapter(torch.nn.Module):
    def __init__(self, core_model):
        super().__init__()
        self.core = core_model

    def forward(self, x):
        y = self.core(x)
        if isinstance(y, (tuple, list)):
            y = y[-1]
        return y

In [124]:
from ultralytics import YOLO
from art.estimators.classification import PyTorchClassifier

yolo  = YOLO(MODEL_PATH)
for p in yolo.model.parameters():
    p.requires_grad_(True)
adapter = YoloClsAdapter(yolo.model).to(device)

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(
    adapter.parameters(),
    lr=LR,
    weight_decay=WEIGHT_DECAY
)

classifier = PyTorchClassifier(
    model=adapter,
    loss=loss,
    optimizer=optimizer,
    nb_classes=10,
    input_shape=(3, IMG_SIZE, IMG_SIZE),
    clip_values=(0.0, 1.0),
    device_type=device
)

В качестве генератора Adversarial examples выберем даже более сильный метод, чем FGSM из предыдущей работы - ProjectedGradientDescent. Это обеспечит защиту как от более слабого метода, так и от некоторых более сильных

In [68]:
from art.attacks.evasion import ProjectedGradientDescent

pgd_attack = ProjectedGradientDescent(
    estimator=classifier,
    norm=np.inf,
    eps=EPS,
    eps_step=EPS_STEP,
    max_iter=MAX_PGD_ITERS,
    targeted=False,
    verbose=False
)

Наконец, создадим наш класс для адверсариального обучения и дообучим нашу модель на 10 эпохах

In [110]:
from art.data_generators import PyTorchDataGenerator
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

CHUNK = 5_000
BATCH_SIZE = 128
EPOCHS     = 5
SHUFFLE_DS = False

torch_transforms = yolo.model.transforms
train_ds = ImageFolder(root=DATA_DIR / "train",
                       transform=torch_transforms)
train_loader = DataLoader(train_ds,
                          batch_size=BATCH_SIZE,
                          shuffle=SHUFFLE_DS,
                          num_workers=4,
                          pin_memory=True)
train_gen = PyTorchDataGenerator(iterator=train_loader,
                                 size=len(train_ds),
                                 batch_size=BATCH_SIZE)

In [125]:
from art.defences.trainer import AdversarialTrainerFBFPyTorch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

classifier._int_labels = True
classifier._reduce_labels = False

adv_trainer = AdversarialTrainerFBFPyTorch(
    classifier,
    eps = 4
)

adv_trainer.fit_generator(        # !!
    generator=train_gen,          # функция, порождающая генератор на каждый epoch
    nb_epochs=1
)

robust_classifier = adv_trainer.get_classifier()
robust_classifier.save('robust_classifier7.pt', 'models')

Adversarial Training FBF - Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

Сохраним обученную модель

In [44]:
robust_classifier.save('robust_classifier2.pt', 'models')

In [40]:
data = torch.load('models/robust_classifier2.pt.model')
data.keys

<function OrderedDict.keys>

In [53]:
from ultralytics import YOLO
import torch

yolo = YOLO(MODEL_PATH)
adapter = YoloClsAdapter(yolo.model).to(device)
adapter.load_state_dict(torch.load('models/robust_classifier2.pt.model'))
robust_classifier = PyTorchClassifier(
    model=adapter,
    loss=torch.nn.CrossEntropyLoss(),
    nb_classes=10,
    input_shape=(3, IMG_SIZE, IMG_SIZE),
    clip_values=(0.0, 1.0),
    device_type=device
)

Теперь сравним качество изначальной и дообученной моделей как на чистой, так и на adv. выборках

In [19]:
from art.estimators.classification import PyTorchClassifier

basic_yolo  = YOLO(MODEL_PATH)
adapter = YoloClsAdapter(basic_yolo.model).to(device).eval()

loss = torch.nn.CrossEntropyLoss()

basic_cls = PyTorchClassifier(
    model=adapter,
    loss=loss,
    nb_classes=10,
    input_shape=(3, IMG_SIZE, IMG_SIZE),
    clip_values=(0.0, 1.0),
    device_type=device
)

In [107]:
from PIL import Image

N_SAMPLES = 1000

all_files = list((DATA_DIR / 'test').glob("*/*.png"))
assert all_files, f"Ничего не найдено в {DATA_DIR}"

idx_files = random.sample(all_files, N_SAMPLES) # Выбираем N_SAMPLES картинок

x, y = [], []
for fp in idx_files:
    img = Image.open(fp).convert("RGB")
    arr = basic_yolo.model.transforms(img)
    x.append(arr)
    y.append(int(fp.parent.name))

clean_x = np.stack(x, axis=0)
clean_y = np.asarray(y)
clean_y

array([5, 3, 2, 3, 2, 1, 9, 8, 8, 5, 7, 2, 1, 0, 2, 3, 9, 3, 2, 5, 2, 7, 7, 8, 3, 1, 5, 1, 1, 9, 1, 1, 8, 5, 5, 7, 2, 2, 5, 1, 8, 7, 1, 2, 3, 1, 5, 4, 7, 6, 1, 7, 3, 5, 3, 0, 4, 3, 5, 0, 9, 8, 3, 0, 4, 3, 6, 9, 0, 3, 2, 1, 5, 2, 2, 5, 3, 1, 2, 1, 9, 2, 1, 9, 6, 5, 5, 5, 1, 2, 7, 2, 3, 2, 1, 2, 1, 6, 6, 9, 2, 2, 0, 0,
       9, 3, 0, 4, 8, 2, 4, 6, 1, 8, 1, 2, 2, 1, 9, 8, 4, 0, 6, 5, 5, 7, 1, 3, 4, 8, 6, 3, 1, 1, 2, 1, 7, 1, 2, 0, 5, 2, 2, 2, 0, 1, 3, 1, 7, 9, 2, 2, 7, 3, 4, 5, 1, 5, 7, 2, 2, 5, 2, 6, 1, 2, 0, 8, 4, 6, 3, 4, 2, 5, 3, 5, 6, 4, 1, 4, 9, 2, 2, 4, 2, 1, 3, 6, 0, 8, 6, 5, 5, 3, 1, 2, 6, 1, 5, 2, 6, 2, 3, 1, 9, 2, 2, 4,
       3, 4, 6, 1, 5, 1, 5, 0, 1, 6, 8, 9, 1, 4, 4, 4, 5, 1, 5, 5, 4, 4, 0, 2, 2, 1, 0, 4, 3, 9, 2, 0, 2, 1, 5, 7, 8, 0, 0, 1, 4, 8, 2, 6, 1, 9, 1, 9, 0, 4, 5, 9, 2, 7, 0, 3, 9, 1, 4, 1, 8, 5, 1, 1, 5, 2, 2, 3, 3, 5, 6, 3, 3, 0, 5, 9, 3, 1, 9, 9, 9, 0, 6, 8, 2, 2, 2, 1, 2, 1, 5, 2, 4, 2, 4, 1, 2, 2, 1, 7, 3, 2, 7, 1,
       4, 3, 1, 2, 5, 9, 5, 1, 9, 2, 1, 4, 

In [44]:
from art.attacks.evasion import FastGradientMethod

EPS = 8/255

# Генерируем FGSM по изначальной модели
basic_fgsm = FastGradientMethod(estimator=basic_cls, eps=EPS)
basic_x_fgsm  = basic_fgsm.generate(x=clean_x)

# FGSM по защищённой модели
robust_fgsm = FastGradientMethod(estimator=robust_classifier, eps=EPS)
robust_x_fgsm  = basic_fgsm.generate(x=clean_x)

In [102]:
train_gen = ChunkedGenerator(
    DATA_DIR / "test",
    chunk=1000,
    batch_size=1000,
    shuffle=False,
)
x, y = train_gen.get_batch()

In [126]:
from sklearn.metrics import accuracy_score

accuracy_score(clean_y, robust_classifier.predict(clean_x).argmax(1))

0.068

In [121]:
import pandas as pd

# ❶ Cводим всё в словари --------------------------------------------------------
tests   = dict(  # порядок сохранится как объявлен
    clean_train   = clean_x,
    basic_x_fgsm  = basic_x_fgsm,
    robust_x_fgsm = robust_x_fgsm
)
models  = dict(basic = basic_cls, robust = robust_classifier)

# ❷ Предсказания всех (model × test) за один проход ----------------------------
preds = {
    (m, t): mdl.predict(x).argmax(1)          # argmax → метки классов
    for m, mdl in models.items()
    for t, x   in tests.items()
}

cols = ["basic_accuracy", "basic_ASR", "robust_accuracy", "robust_ASR"]
df = pd.DataFrame(index=tests, columns=cols, dtype="float32")

for m in models:
    clean_pred = preds[(m, "clean_train")]
    acc_col = f"{m}_accuracy"
    asr_col = f"{m}_ASR"

    for t in tests:
        adv_pred = preds[(m, t)]
        accuracy = np.mean(adv_pred == clean_y)
        if t == "clean_train":
            asr = 0
        else:
            asr = np.mean((clean_pred == clean_y) & (adv_pred != clean_y))

        df.at[t, acc_col] = accuracy
        df.at[t, asr_col] = asr

df

C:\Users\ok__\AppData\Local\Temp\ipykernel_19856\1630435535.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.973' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.at[t, acc_col] = accuracy
C:\Users\ok__\AppData\Local\Temp\ipykernel_19856\1630435535.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.804' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.at[t, asr_col] = asr
C:\Users\ok__\AppData\Local\Temp\ipykernel_19856\1630435535.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.191' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.at[t, acc_col] = accuracy


,basic_accuracy,basic_ASR,robust_accuracy,robust_ASR
clean_train,0.973,0.000,0.191,0.0
basic_x_fgsm,0.172,0.804,0.191,0.0
robust_x_fgsm,0.172,0.804,0.191,0.0
